## Defining Random Variables in =python=                        :code_example:



### Introduction



The distinguishing feature of variables in a field such as the
reals or the complex plane is their *value*; the distinguishing
feature of random variables is their *distribution*.  The `python`
package `scipy.stats` is well-engineered and offers many different
distributions, and tools to construct others, while the package
`pacal` is perhaps less well engineered, but defines arithmetic
operations over random variables which allows for more elegant
semantics.



In [1]:
from scipy.stats import distributions as iid

class RandomVariable:

There are two main classes of random variables to consider: discrete
   and continuous.  The distinction is worth drawing because different
   classes are handled differently in many mathematical operations.  

For example, here we instantiate a scalar continuous $\rvx$:



In [1]:
x = iid.norm()

And here we instantiate a discrete random variable which is defined
over an event space $\{-1,0,1\}$ with corresponding probabilities $(1/3,1/2,1/6)$:



In [1]:
from scipy.stats._distn_infrastructure import rv_sample, rv_frozen
from scipy.stats import distributions as iid

class GenericDiscreteRandomVariable(rv_sample,iid.rv_discrete):
    def __new__(cls, *args, **kwds):
        return super(rv_sample, cls).__new__(cls)

    __init__ = rv_sample.__init__


class SpecialDiscreteRandomVariable(rv_frozen,iid.rv_discrete):
    def __new__(cls, *args, **kwds):
        f = args[0]
        return super().__new__(cls,f)

    def __init__(self,*args,**kwds):
        dist = args[0].dist
        self = dist
        self.dist = dist
        self.args = args
        self.kwds = kwds

class DiscreteRandomVariable(GenericDiscreteRandomVariable,SpecialDiscreteRandomVariable):
    def __new__(cls, *args, **kwds):
        print(args)
        if 'values' in kwds or hasattr(args[0],'pk'):
            return super(GenericDiscreteRandomVariable,cls).__new__(cls, *args, **kwds)
        elif hasattr(args[0],'dist'):
            return super(SpecialDiscreteRandomVariable,cls).__new__(cls, *args, **kwds)
        
    def __init__(self, *args, **kwds):
        if 'values' in kwds or hasattr(args[0],'pk'):
            super(GenericDiscreteRandomVariable,self).__init__(self, *args, **kwds)
        elif hasattr(self,'dist'):
            f = args[0]
            super(SpecialDiscreteRandomVariable,self).__init__(self, *args, **kwds)

In [1]:
from scipy.stats._distn_infrastructure import rv_sample, rv_frozen
from scipy.stats import distributions as iid
from numpy import floor

class DiscreteRandomVariable(rv_sample,iid.rv_discrete):

    def __new__(cls, *args, **kwds):
        return super(rv_sample, cls).__new__(cls)

    def __init__(self,s):
        if hasattr(s,'pk'):
            self.type = "generic"
        elif hasattr(s,'dist'):
            self.type = 'special'
            
        super(DiscreteRandomVariable, self).__init__(name="DiscreteRandomVariable")

    def _special_pmf(self,x):
        try:
            return self.pk[self.xk.index(x)]
        except ValueError:
            return 0

    def _pmf(self,x):
        return _special_pmf(self,x)

In [1]:
Omega = (-1,0,1)
Pr = (1/3.,1/2.,1/6.)

s = DiscreteRandomVariable(values=(Omega,Pr))

Now, here are some things we can do with these random variables.
 First, the continuous  $\rvx$:



In [1]:
print("E(x) = %6.4f" % x.mean())
print()
print("Some (central) moments of x:")
print([(m,x.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % x.interval(0.95))
print()
print(x.cdf(0),x.pdf(0))

:results:
 E(x) = 0.0000

 Some (central) moments of x:
 [(1, 0.0), (2, 1.0), (3, 0.0), (4, 3.0)]

 95% confidence interval: (-1.959964,1.959964)

 0.5 0.3989422804014327
 :end:

Next, the discrete r.v., \rv{s}:



In [1]:
print("E(s) = %6.4f" % s.mean())
print()
print("Some moments of s:")
print([(m,s.moment(m)) for m in [1,2,3,4]])
print()
print("95%% confidence interval: (%f,%f)" % s.interval(0.95))
print()
# Note! Not pdf, but pmf for discrete rv.
print(s.cdf(0),s.pmf(0))

If we want *realizations* of these random variables:



In [1]:
N=3
print(x.rvs(N)) # N realizations; no longer random

### Convolutions



We&rsquo;d like to be able to combine different random variables, say
by addition, yielding a new random variable.  For instance, we&rsquo;d like
to be able to construct



In [1]:
y = x + s

But this fails.  Can you explain why?  What do you suppose the cdf of
$\rvy$ looks like?  Does it have a density, or does the addition of a
random variable that *lacks* a density ($\rv{s}$) to a random variable
that has one ($\rvx$) mess things up?



#### Convolution of Continuous & Discrete Random variables



In [1]:
from scipy.stats import distributions as iid

# Code to convolve a random variable with a pmf and another having a cdf
# Exploits =scipy.stats= base rv_continuous class.

class ConvolvedContinuousAndDiscrete(iid.rv_continuous):

    """Convolve (add) a continuous rv x and a discrete rv s,
       returning the resulting cdf."""

    def __init__(self,f,s):
        self.continuous_rv = f
        self.discrete_rv = s
        super(ConvolvedContinuousAndDiscrete, self).__init__(name="ConvolvedContinuousAndDiscrete")
        
    def _cdf(self,z):
        F=0
        s = self.discrete_rv
        x = self.continuous_rv
        
        for k in range(len(s.xk)):
            F = F + x.cdf(z-s.xk[k])*s.pk[k]
        return F

    def _pdf(self,z):
        f=0
        s = self.discrete_rv
        x = self.continuous_rv
        
        for k in range(len(s.xk)):
            f = f + x.pdf(z-s.xk[k])*s.pk[k]
        return f


# Create new convolved rv:
y = ConvolvedContinuousAndDiscrete(x,s)

In [1]:
import plotly.graph_objects as go
import numpy as np

X = np.linspace(-4,4,100).tolist()

fig = go.Figure(data=go.Scatter(x=X, y=[y.pdf(z) for z in X]))
fig.show()

### Exercise



Prove that $\rvy$ is continuous (in the sense that it has a density),
    as suggested by the figure *or* establish that the figure is
    wrong or misleading.

